👗В этом ноутбуке проверяется, как нейросети  (**BERT и ResNet**) справляются с задачей классификации товаров по описанию.

🏆Лучший результат (**0.84**): модель `cointegrated/rubert-tiny2`, дообученная только на описании товаров.

#Подготовка

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!git clone https://github.com/anna-marshalova/item-classification 
%cd item-classification

In [4]:
!unzip -qq '/content/drive/MyDrive/internship_2023.zip'

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.1 MB/s eta 0:00:00


Для дальнейших экспериментов будем использовать модель [cointegrated/rubert-tiny2](https://huggingface.co/cointegrated/rubert-tiny). Она была выбрана благодаря своей легкости: с более тяжелыми моделями GPU в Google Colab не справлялся.

In [ ]:
MODEL_NAME = 'cointegrated/rubert-tiny2'

Загружаем и предобрабатываем обучающие данные.

In [ ]:
from dataset import Dataset
train_dataset = Dataset('train.parquet', MODEL_NAME)

Praparing dataset...


  0%|          | 0/91120 [00:00<?, ?it/s]

Создаем Data Loader'ы, чтобы разделить данные на батчи.

In [ ]:
from data_loader import get_train_loaders
train_loader, val_loader = get_data_loaders(train_dataset, num_workers=2)

#Обучение

Классы в данных несбалансированы: в некоторых категориях несколько сотен товаров, а в некоторых в всего 1. Чтобы бороться с этой проблемой, определим веса классов - так модель будет обращать больше внимания на малочисленные классы.

In [ ]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
class_weights=compute_class_weight('balanced',classes=np.array([*train_dataset.category2class.keys()]),y=train_dataset.df.category_id)
class_weights=torch.tensor(class_weights)

Инициализируем класс для обучения модели.

In [ ]:
from trainer import Trainer
from utils import get_model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
experiment_name = 'text_image' #определим название эксперимента, чтобы сохранять веса и метрики
model = get_model(experiment_name, device, train_dataset.num_labels, MODEL_NAME, load_weights=True, shop_id_vector_size = len(train_dataset.shop_id2categorical))
trainer = Trainer(model, device, experiment_name, class_weights = class_weights)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is

Loading model from /content/drive/MyDrive/KazanExpress/weights/text_image_model.pt


Запускаем обучение

In [ ]:
trainer.train(train_loader, val_loader, 5)

#Оценка

Оценим качество классификации на валидационных данных по метрике weighted F1.

Модель, обученная только на текстовых данных (описане + название)

In [ ]:
from evaluate import evaluate
from utils import save_metrics
experiment_name = 'text'
model = get_model(experiment_name, device, train_dataset.num_labels, MODEL_NAME, load_weights=True, shop_id_vector_size = len(train_dataset.shop_id2categorical))
true, preds = evaluate(model, val_loader, device)
save_metrics(experiment_name, true, preds)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from /content/drive/MyDrive/KazanExpress/weights/text_model.pt


  0%|          | 0/285 [00:00<?, ?it/s]

F1 for batch 284: 0.8333333333333334Total F1 score: 0.8411705908443583
Metrics saved to /content/drive/MyDrive/KazanExpress/metrics/text.csv


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

Модель, обученная на текстовых данных и категориальных признаках (id магазина).

In [ ]:
from evaluate import evaluate
from utils import save_metrics
experiment_name = 'text_categorical'
model = get_model(experiment_name, device, train_dataset.num_labels, MODEL_NAME, load_weights=True, shop_id_vector_size = len(train_dataset.shop_id2categorical))
true, preds = evaluate(model, val_loader, device)
save_metrics(experiment_name, true, preds)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from /content/drive/MyDrive/KazanExpress/weights/text_categorical_model.pt


  0%|          | 0/285 [00:00<?, ?it/s]

F1 for batch 284: 0.7916666666666666Total F1 score: 0.8267373339363528
Metrics saved to /content/drive/MyDrive/KazanExpress/metrics/text_categorical.csv


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

Модель, обученная на текстовых данных и изображениях товаров

In [ ]:
from evaluate import evaluate
from utils import save_metrics
experiment_name = 'text_image'
model = get_model(experiment_name , device, train_dataset.num_labels, MODEL_NAME, load_weights=True, shop_id_vector_size = len(train_dataset.shop_id2categorical))
true, preds = evaluate(model, val_loader, device)
save_metrics(experiment_name, true, preds)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is

Loading model from /content/drive/MyDrive/KazanExpress/weights/text_image_model.pt


  0%|          | 0/285 [00:00<?, ?it/s]

F1 for batch 284: 0.8541666666666666Total F1 score: 0.7588139360412594
Metrics saved to /content/drive/MyDrive/KazanExpress/metrics/text_image.csv


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

Итак, лучший результат показала модель, обученная только на текстах. Это может быть связано с:
1. большим количеством магазинов в датасете (более 10000). вектор магазина имеет слишком большую размерность, при этом, проходя через линейный слой, он сжимается до 16 значений. возможно, часть информации на этом этапе теряется, поэтому добавление этого признака не улучшает классификацию. 
2. особенностями валидационной выборки. разница между метриками для моделей с категориальными признаками и без составляет всего 0.015, поэтому ее, возможно, нельзя считать значимой. на другой валидационной выборке метрики могли бы отличаться в другую сторону.
3. особенностями изображений товаров. проводился эксперимент с обучением модели только на картинках, и результаты даже на обучающих данных не были впечатляющими (около 0.2 после 3 эпох). возможно, resnet18 слишком мала для этой задачи.


#Предсказание на тестовых данных

Используем лучшую модель для получения предсказаний на тестовых данных и сохраним результаты.

In [ ]:
test_dataset = Dataset('test.parquet', MODEL_NAME, mode='test')

Praparing dataset...


  0%|          | 0/16860 [00:00<?, ?it/s]

In [ ]:
from torch.utils.data import DataLoader
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
from predict import predict
model = get_model('text', device, train_dataset.num_labels, MODEL_NAME, load_weights=True, shop_id_vector_size = len(train_dataset.shop_id2categorical))
result_df = predict(model, test_loader, device, train_dataset.category2class)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from /content/drive/MyDrive/KazanExpress/weights/text_model.pt


  0%|          | 0/264 [00:00<?, ?it/s]

In [ ]:
assert len(result_df) == len(test_dataset.df)

In [ ]:
result_df

,product_id,predicted_category_id
0,1997646,13495
1,927375,14922
2,1921513,2803
3,1668662,15075
4,1467778,13887
...,...,...
16855,1914264,11645
16856,1310569,12357
16857,978095,13651
16858,797547,2740


In [ ]:
result_df.to_parquet('result.parquet')